In [6]:
import pandas as pd
import numpy as np

In [7]:
input_limit = 100
missing_data = '?'
have_header = False
character_missing_data = "missing"  # For now I have used most frequent character for missing values
result_column = 1
epoches = 10000
learning_rate = 0.001

In [8]:
def get_col(df, arg, limit):
    return df[arg][:limit].to_numpy()

def init():
    try:
        if (have_header):
            df = pd.read_csv("crx.xls")
        else:
            df = pd.read_csv("crx.xls", header = None)
    except Exception as e:
        print("file not found\n")
        print(e+"\n")
    return df

def get_features(df, n, limit):
    datas = []
    datas = df.iloc[:limit, :n]
    return np.array(datas)

def get_labels(df, n,limit):
    datas = []
    for i in range(result_column):
        datas.append(get_col(df, n-i, limit))
    return np.array(datas)

def solve_missing_data(features):
    n = features.shape[1]
    for i in range(n):
        zero_count = (features[:,i] == missing_data).sum()
        if (zero_count > 0):
            index_list = np.where((features[:,i] == missing_data))[0]
            # TODO: I should change 0 index, It might be zero itself
            if (isinstance(features[0, i], (int, float))):
                avg = np.mean(features[:, i])
                for j in index_list:
                    features[j, i] = avg
            else:
                values, counts = np.unique(features[:, i], return_counts=True)
                most_frequent = values[np.argmax(counts)]
                for j in index_list:
                    features[j, i] = most_frequent
    return features

def features_convert_to_number(features):
    m, n = features.shape
    for i in range (n):
        data = features[:, i]
        if (isinstance(data[0], (int, float))):
            continue
        unique_data = np.unique(data)
        len_unique_data = len(unique_data)
        if (len_unique_data == 2):
            for j in range (m):
                if (data[j] == unique_data[0]):
                    data[j] = 0
                else:
                    data[j] = 1
        elif (len_unique_data > 2):
            num_arr = np.arange(1, len_unique_data+1, dtype=float)
            min = np.min(num_arr)
            max = np.max(num_arr)
            distance = max - min
            for j in range(len_unique_data):
                num_arr[j] -= min
                num_arr[j] /= distance
            for j in range (m):
                idx = np.where(unique_data == data[j])[0][0]
                data[j] = num_arr[idx]

        features[:, i] = data
    features = features.astype(float)
    return features

def labels_convert_to_number(lables):
    n = lables.shape[1]
    for i in range (n):
        if (lables[0][i] == '+'):
            lables[0][i] = 1
        else:
            lables[0][i] = 0
    lables = lables.astype(float).flatten()
    return lables

def init_weights_bias(ntehta):
    weights = np.zeros(ntehta)
    bias = 0
    return weights, bias

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logreg_compute_gradiant(features, lables, thetas, bias):
    m, n = features.shape
    z = np.dot(features, thetas) + bias
    predictions = sigmoid(z)
    dw = (1/m) * np.dot(features.T, (predictions - lables))
    db = (1/m) * np.sum(predictions - lables)
    thetas = thetas - learning_rate * dw
    bias = bias - learning_rate * db
    return thetas, bias

def logreg_compute_cost(features, lables, thetas, bias):
    m = len(lables)
    z = np.dot(features, thetas) + bias
    prediction = sigmoid(z)
    epsilon = 1e-15
    prediction = np.clip(prediction, epsilon, 1 - epsilon)
    cost = -np.mean(lables * np.log(prediction) + (1-lables) * np.log(1-prediction))
    return cost

def logistic_regression(features, lables, nrows, thetas, bias):
    ncols = len(features)
    cost_history = []
    for epoch in range (epoches):
        thetas, bias = logreg_compute_gradiant(features, lables, thetas, bias)
        cost = logreg_compute_cost(features, lables, thetas, bias)
        cost_history.append(cost)
    return thetas, bias, cost_history

def get_test_inputs(df, n, limit):
    datas = []
    datas = df.iloc[limit:, :n]
    datas = np.array(datas)
    return datas

def get_test_results(df, n, limit):
    datas = []
    datas = df.iloc[limit:, n]
    datas = np.array(datas)
    return datas

def tresults_convert_to_number(test_results):
    n = len(test_results)
    for j in range (n):
        if (test_results[j] == '+'):
            test_results[j] = 1
        else:
            test_results[j] = 0
    test_results = np.array(test_results)
    return test_results

def predict(x, y, thetas, bias):
    n, m = x.shape
    ylen = len(y)
    z = np.dot(x, thetas) + bias
    predictions = sigmoid(z)
    plen = len(predictions)
    for i in range(plen):
        if (predictions[i] >= 0.5):
            predictions[i] = 1
        else:
            predictions[i] = 0
    success = 0
    for i in range (plen):
        if (predictions[i] == y[i]):
            success += 1
    return (success / ylen)

def main():
    df = init()
    ninput_data = df.shape[0] - input_limit
    if (ninput_data < 0):
        ninput_data = df.shape[0]
        ninput_test = 0
    else:
        ninput_test = input_limit
    nfeaurescols = df.shape[1] - result_column
    features = get_features(df, nfeaurescols, ninput_data)
    labels = get_labels(df, df.shape[1]-1, ninput_data)
    features = solve_missing_data(features)
    features = features_convert_to_number(features)
    labels = labels_convert_to_number(labels)
    theta_arr, bias = init_weights_bias(nfeaurescols)
    theta_arr, bias, cost_history = logistic_regression(features, labels, ninput_data, theta_arr, bias)
    if (ninput_test):
        test_inputs = get_test_inputs(df, nfeaurescols, ninput_data)
        test_results = get_test_results(df, nfeaurescols, ninput_data)
        test_results = tresults_convert_to_number(test_results)
        test_inputs = solve_missing_data(test_inputs)
        test_inputs = features_convert_to_number(test_inputs)
        sucess_percent = predict(test_inputs, test_results, theta_arr, bias)
        print(sucess_percent)


In [24]:
df = pd.read_csv("crx.csv")

In [26]:
df.head(10)


,b,30.83,0,u,g,w,v,1.25,t,t.1,1,f,g.1,202,0.1,+
0,a,58.67,4.460,u,g,q,h,3.040,t,t,6,f,g,43,560,+
1,a,24.5,0.500,u,g,q,h,1.500,t,f,0,f,g,280,824,+
2,b,27.83,1.540,u,g,w,v,3.750,t,t,5,t,g,100,3,+
3,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,120,0,+
4,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,360,0,+
5,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,164,31285,+
6,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,80,1349,+
7,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,180,314,+
8,b,42.5,4.915,y,p,w,v,3.165,t,f,0,t,g,52,1442,+
9,b,22.08,0.830,u,g,c,h,2.165,f,f,0,t,g,128,0,+


In [29]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   b       689 non-null    object 
 1   30.83   689 non-null    object 
 2   0       689 non-null    float64
 3   u       689 non-null    object 
 4   g       689 non-null    object 
 5   w       689 non-null    object 
 6   v       689 non-null    object 
 7   1.25    689 non-null    float64
 8   t       689 non-null    object 
 9   t.1     689 non-null    object 
 10  1       689 non-null    int64  
 11  f       689 non-null    object 
 12  g.1     689 non-null    object 
 13  202     689 non-null    object 
 14  0.1     689 non-null    int64  
 15  +       689 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.3+ KB


In [33]:
df['b'].value_counts()


,count
b,
b,467
a,210
?,12


In [54]:
for col in df.columns:
  df[col] = df[col].replace('?', np.nan)

In [55]:
rows_with_nan = df[df.isna().any(axis=1)]
print(rows_with_nan)

       b  30.83       0    u    g    w    v    1.25  t t.1  1  f g.1   202  \
70     b  34.83   4.000    u    g    d   bb  12.500  t   f  0  t   g   NaN   
82     a    NaN   3.500    u    g    d    v   3.000  t   f  0  t   g   300   
85     b    NaN   0.375    u    g    d    v   0.875  t   f  0  t   s   928   
91     b    NaN   5.000    y    p   aa    v   8.500  t   f  0  f   g     0   
96     b    NaN   0.500    u    g    c   bb   0.835  t   f  0  t   s   320   
201    b  24.83   2.750    u    g    c    v   2.250  t   t  6  f   g   NaN   
205    a  71.58   0.000  NaN  NaN  NaN  NaN   0.000  f   f  0  f   p   NaN   
242    a  18.75   7.500    u    g    q    v   2.710  t   t  5  f   g   NaN   
247  NaN   24.5  12.750    u    g    c   bb   4.750  t   t  2  f   g    73   
253    b    NaN   0.625    u    g    k    v   0.250  f   f  0  f   g   380   
269    b  37.58   0.000  NaN  NaN  NaN  NaN   0.000  f   f  0  f   p   NaN   
277    b  24.58  13.500    y    p   ff   ff   0.000  f   f  0  f